In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

## Python Google calendar quickstart

In [4]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=67229562181-fdj73daqo2rfpvvv6lfquf26s8dq926s.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64580%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=1hazhAnY38ZVj9tplVwnd3ZLtoApY6&access_type=offline
Getting the upcoming 10 events
2024-01-03T19:30:00+05:30 Weekly Tech capability- AI
2024-01-05T22:30:00+05:30 Marketing Updates
2024-01-10T19:30:00+05:30 Weekly Tech capability- AI
2024-01-12T22:30:00+05:30 Marketing Updates
2024-01-17T19:30:00+05:30 Weekly Tech capability- AI
2024-01-19T22:30:00+05:30 Marketing Updates
2024-01-24T19:30:00+05:30 Weekly Tech capability- AI
2024-01-26T22:30:00+05:30 Marketing Updates
2024-01-31T19:30:00+05:30 Weekly Tech capability- AI
2024-02-02T22:30:00+05:30 Marketing Updates


### Lets see how to create a new meeting

In [9]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

In [48]:
def create_event(service, summary, location, description, start_time, end_time, attendees=None, reminders=None):
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': 'UTC',  # Adjust timezone as needed
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'UTC',
        },
        'attendees': attendees,
        'reminders': reminders,
    }

    try:
        created_event = service.events().insert(
            calendarId='primary', body=event).execute()
        print(f'Event created: {created_event.get("htmlLink")}')
    except HttpError as e:
        print(f'Error creating event: {e}')

In [49]:
summary = 'Google I/O 2015'
location = '800 Howard St., San Francisco, CA 94103'
description = 'A chance to hear more about Google\'s developer products.'
start_time = '2023-12-30T16:30:00+05:30'
end_time = '2023-12-30T18:30:00+05:30'
service = build('calendar', 'v3', credentials=creds)
create_event(service, summary, location, description, start_time, end_time, attendees, reminders)

Event created: https://www.google.com/calendar/event?eid=Ym52b25uODU1amozaWlka2xrYzVzbW00MGMgYXJ5YW5zYW50b3NocGlsbGFpQG0


In [13]:
def find_event_by_summary(service, summary):
    events_result = service.events().list(
        calendarId='primary', q=summary).execute()
    events = events_result.get('items', [])

    if not events:
        print(f'No events found with summary "{summary}"')
        return None

    for event in events:
        print(f'Summary: {event["summary"]}')
        print(f'Start Time: {event["start"].get("dateTime", event["start"].get("date"))}')
        print(f'End Time: {event["end"].get("dateTime", event["end"].get("date"))}')
        print(f'Location: {event.get("location", "N/A")}')
        print('---')

    return events

In [14]:
summary_to_find = 'Google I/O 2015'
found_events = find_event_by_summary(service, summary_to_find)
found_events

Summary: Google I/O 2015
Start Time: 2023-12-30T16:30:00+05:30
End Time: 2023-12-30T18:30:00+05:30
Location: 800 Howard St., San Francisco, CA 94103
---


[{'kind': 'calendar#event',
  'etag': '"3407777310212000"',
  'id': '51o6u74dpfmlc8k39ch39v2k2c',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=NTFvNnU3NGRwZm1sYzhrMzljaDM5djJrMmMgYXJ5YW5zYW50b3NocGlsbGFpQG0',
  'created': '2023-12-29T22:24:15.000Z',
  'updated': '2023-12-29T22:24:15.106Z',
  'summary': 'Google I/O 2015',
  'description': "A chance to hear more about Google's developer products.",
  'location': '800 Howard St., San Francisco, CA 94103',
  'creator': {'email': 'aryansantoshpillai@gmail.com', 'self': True},
  'organizer': {'email': 'aryansantoshpillai@gmail.com', 'self': True},
  'start': {'dateTime': '2023-12-30T16:30:00+05:30', 'timeZone': 'UTC'},
  'end': {'dateTime': '2023-12-30T18:30:00+05:30', 'timeZone': 'UTC'},
  'iCalUID': '51o6u74dpfmlc8k39ch39v2k2c@google.com',
  'sequence': 0,
  'attendees': [{'email': 'sbrin@example.com',
    'responseStatus': 'needsAction'},
   {'email': 'lpage@example.com', 'responseStatus': 'needsAction

In [28]:
from datetime import datetime, timedelta

def get_events_on_date(service, target_date):
    # Set the start and end time for the specified date
    start_time = target_date.isoformat() + 'T00:00:00Z'
    end_time = (target_date + timedelta(days=1)).isoformat() + 'T00:00:00Z'

    # Query events for the specified date
    events_result = service.events().list(
        calendarId='primary', timeMin=start_time, timeMax=end_time).execute()
    events = events_result.get('items', [])

    return events


In [29]:
target_date = datetime(2023, 12, 30).date()
events=get_events_on_date(service, target_date)
events

[{'kind': 'calendar#event',
  'etag': '"3407779393380000"',
  'id': 'dp24bsll31ur3mjq38ks1pphck',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=ZHAyNGJzbGwzMXVyM21qcTM4a3MxcHBoY2sgYXJ5YW5zYW50b3NocGlsbGFpQG0',
  'created': '2023-12-29T22:41:36.000Z',
  'updated': '2023-12-29T22:41:36.690Z',
  'summary': 'Google I/O 2015',
  'description': "A chance to hear more about Google's developer products.",
  'location': '800 Howard St., San Francisco, CA 94103',
  'creator': {'email': 'aryansantoshpillai@gmail.com', 'self': True},
  'organizer': {'email': 'aryansantoshpillai@gmail.com', 'self': True},
  'start': {'dateTime': '2023-12-30T16:30:00+05:30', 'timeZone': 'UTC'},
  'end': {'dateTime': '2023-12-30T18:30:00+05:30', 'timeZone': 'UTC'},
  'iCalUID': 'dp24bsll31ur3mjq38ks1pphck@google.com',
  'sequence': 0,
  'attendees': [{'email': 'sbrin@example.com',
    'responseStatus': 'needsAction'},
   {'email': 'lpage@example.com', 'responseStatus': 'needsAction

In [31]:
def get_todays_date():
    today_date = datetime.now().date()
    return today_date

# Example usage:
today = get_todays_date()
today

datetime.date(2023, 12, 30)

In [33]:
target_date = datetime(2023, 12, 30).date()
target_date

datetime.date(2023, 12, 30)

In [43]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
import datetime

class datein(BaseModel):
    query: datetime.date = Field(description="should be a date in datetime.date")


@tool("get_events_on_date", args_schema=datein, return_direct=True)
def get_events_on_date(target_date: datetime.date) ->:
    """Used to find events on the calendar for a specific date input is the date"""
    return target_date

In [44]:
print(get_events_on_date.name)
print(get_events_on_date.description)
print(get_events_on_date.args)
print(get_events_on_date.return_direct)

get_events_on_date
get_events_on_date(target_date: datetime.date) -> list - Used to find events on the calendar for a specific date input is the date
{'query': {'title': 'Query', 'description': 'should be a date in datetime.date', 'type': 'string', 'format': 'date'}}
True


In [45]:
class EventInput(BaseModel):
    summary: str = Field(description="Event summary")
    location: str = Field(description="Event location")
    description: str = Field(description="Event description")
    start_time: str = Field(description="Start time for the event")
    end_time: str = Field(description="End time for the event")
    attendees: list = Field(default=None, description="List of attendees", title="Attendees")
    reminders: dict = Field(default=None, description="Event reminders", title="Reminders")

@tool("create-event-tool", args_schema=EventInput, return_direct=True)
def create_event(summary: str, location: str, description: str, start_time: str, end_time: str, attendees=None, reminders=None) -> str:
    """Create a new event."""
    # Your implementation here
    return f"Event created: {summary}"

In [47]:
print(create_event.name)
print(create_event.description)
print(create_event.args)
print(create_event.return_direct)

create-event-tool
create-event-tool(summary: str, location: str, description: str, start_time: str, end_time: str, attendees=None, reminders=None) -> str - Create a new event.
{'summary': {'title': 'Summary', 'description': 'Event summary', 'type': 'string'}, 'location': {'title': 'Location', 'description': 'Event location', 'type': 'string'}, 'description': {'title': 'Description', 'description': 'Event description', 'type': 'string'}, 'start_time': {'title': 'Start Time', 'description': 'Start time for the event', 'type': 'string'}, 'end_time': {'title': 'End Time', 'description': 'End time for the event', 'type': 'string'}, 'attendees': {'title': 'Attendees', 'description': 'List of attendees', 'type': 'array', 'items': {}}, 'reminders': {'title': 'Reminders', 'description': 'Event reminders', 'type': 'object'}}
True


In [54]:
def create_event(summary, location, description, start_time, end_time, attendees=None, reminders=None):
    # Your implementation here
    print(f"Event created: {summary}, {location}, {description}, {start_time}, {end_time}, {attendees}, {reminders}")

def create_string(input_string):
    # Split the input string into individual fields
    fields = [field.strip() for field in input_string.split(',')]

    # Call the function with the created variables
    create_event(**{field: None for field in fields})

# Example usage:
input_string = "summary, location, description, start_time, end_time"
create_string(input_string)


Event created: None, None, None, None, None, None, None
